In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.20.0


In [3]:
TRAIN_DIR = "data/train"   
TEST_DIR  = "data/test"

IMG_HEIGHT = 64
IMG_WIDTH  = 64
BATCH_SIZE = 32

In [4]:
# 1. Update the DataGenerator to include a validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Sets aside 20% of training data for validation
)

# 2. Training generator (using the 'training' subset)
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'     
)

# 3. Validation generator (using the 'validation' subset from the same folder)
test_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,            # Pointing to 'train' because it has the class folders
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',  
    shuffle=False
)

num_classes = train_generator.num_classes
print("Number of flower classes:", num_classes)

Found 5283 images belonging to 102 classes.
Found 1269 images belonging to 102 classes.
Number of flower classes: 102


In [5]:
cnn = Sequential()

# Conv Block 1
cnn.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu',
               input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
cnn.add(MaxPooling2D(pool_size=(2,2)))

# Conv Block 2
cnn.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

# Conv Block 3
cnn.add(Conv2D(filters=128, kernel_size=(3,3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))

cnn.add(Flatten())

# Dense Layers (ANN)
cnn.add(Dense(128, activation='relu'))
cnn.add(Dropout(0.5))

# Output layer for flower classes
cnn.add(Dense(num_classes, activation='softmax'))

cnn.summary()


c:\Users\Adiraj\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 12, 12, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 6, 6, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 102)            │        13,158 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 696,358 (2.66 MB)

 Trainable params: 696,358 (2.66 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = cnn.fit(
    train_generator,
    epochs=30,
    validation_data=test_generator, 
    callbacks=[early_stop]
)


Epoch 1/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 82s 475ms/step - accuracy: 0.0339 - loss: 4.4502 - val_accuracy: 0.0623 - val_loss: 4.1108
Epoch 2/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 114s 690ms/step - accuracy: 0.0664 - loss: 4.0706 - val_accuracy: 0.0788 - val_loss: 3.8083
Epoch 3/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 116s 698ms/step - accuracy: 0.0797 - loss: 3.8485 - val_accuracy: 0.1214 - val_loss: 3.5834
Epoch 4/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 108s 649ms/step - accuracy: 0.1062 - loss: 3.6699 - val_accuracy: 0.1678 - val_loss: 3.4779
Epoch 5/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 118s 708ms/step - accuracy: 0.1247 - loss: 3.5381 - val_accuracy: 0.1883 - val_loss: 3.3270
Epoch 6/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 119s 717ms/step - accuracy: 0.1493 - loss: 3.3923 - val_accuracy: 0.2096 - val_loss: 3.1637
Epoch 7/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 144s 728ms/step - accuracy: 0.1630 - loss: 3.3136 - val_accuracy: 0.2498 - val_loss: 3.0494
Epoch 8/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 117s 704ms/step - accuracy: 0.1936 - 

In [ ]:
cnn.save("final_model.keras")
print("Model saved as final_model.keras")

Model saved as final_model.keras


In [7]:
import numpy as np
from tensorflow.keras.preprocessing import image

loaded_model = tf.keras.models.load_model("final_model.keras")

class_indices = train_generator.class_indices
idx_to_class = {v: k for k, v in class_indices.items()} 

def predict_flower(img_path):
    img = image.load_img(img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
    x = image.img_to_array(img) / 255.0
    x = np.expand_dims(x, axis=0)

    preds = loaded_model.predict(x)
    class_id = int(np.argmax(preds[0]))
    class_name = idx_to_class[class_id]
    confidence = float(preds[0][class_id])

    print(f"Predicted class: {class_name} (confidence: {confidence:.2f})")


c:\Users\Adiraj\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [11]:
predict_flower("3.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
Predicted class: Passion Flower (confidence: 1.00)
